# checking  error log by comparing normal files

In [5]:
from pyspark import SparkContext, SparkConf #spark

In [19]:
from pyspark.sql import SQLContext # for data frame

In [7]:
conf = SparkConf().setAppName("error_machine_for_error_log")

In [160]:
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=error_machine_for_error_log, master=local[2]) created by __init__ at <ipython-input-8-6b825dbb354c>:1 

In [161]:
sqlCtx = SQLContext(sc)

In [211]:
def Get_Different_text(context, compare_folder, list_rdd_key):
    list_not_find = []
    list_rdd_key = {}
    keyword = ""
    try:
        if (len(context.split(" ")) > 4):
            if ("[" in context.split(" ")[4]):
                key_temp = context.split(" ")[4]
                keyword = key_temp.split("[")[0]
                keyword = str(keyword.strip())
                values = context.split(key_temp)[1].split(" ")
                #get hdfs if never used
                if(str(keyword) not in list_rdd_key):
                    list_rdd_key[str(keyword)] = sc.textFile(compare_folder+ str(keyword))
                for val in values:
                    flg_found = 0
                    if (val != ""):
                        for i in range (1,len_compare):
                            #split lines by space 
                            comapre_line = compare[i].split(" ")
                            for ele_com_line in comapre_line:
                                #compare value from each text
                                if (str(val) in str(ele_com_line)):
                                    flg_found = 1
                                    break;
                            if(flg_found == 1):
                                break;
                        if(flg_found == 0):
                            list_not_find.append(str(val))
            else:
                list_not_find.append("[not found[]]: " + str(context))
        else:
            list_not_find.append("[not found, less word]: " + str(context))
    except:
        list_not_find.append("[error happen]: " + str(context))
        print("error happen==="+str(keyword))
    return (keyword, list_not_find, list_rdd_key)

In [217]:
def MapReduce_check(inputPath, outputFolder, compare_folder):
    df_in = sc.textFile(inputPath)
    collect_in = df_in.collect()
    res_list = []
    list_rdd_key = {}
    #looping each line from input text
    for i in range(0,df_in.count()):
        (keyword, not_find_list, list_rdd_key) = Get_Different_text(collect_in[i], compare_folder, list_rdd_key)
        if (not_find_list != []):
            res_list.append((i+1,keyword,not_find_list))
    df = sqlCtx.createDataFrame(res_list, ['line', 'key', 'not_find'])
    res = df.rdd.saveAsTextFile(outputFolder)
    return df

In [218]:
inputPath = '/user/spark/input/log_error'

In [219]:
outputFolder  = '/user/spark/output/find_error'

In [220]:
compare_folder  = '/user/spark/output/log_distinct/'

In [221]:
res =  MapReduce_check(inputPath, outputFolder, compare_folder)

In [199]:
print res.limit(50).show()

+----+--------------+--------------------+
|line|           key|            not_find|
+----+--------------+--------------------+
|   1|       anacron|                  []|
|   2|       anacron|                  []|
|   3|      dhclient|[DHCPREQUEST, of,...|
|   4|      dhclient|[DHCPACK, of, 192...|
|   5|NetworkManager|[<info>, [1484178...|
|   6|NetworkManager|[<info>, [1484178...|
|   7|NetworkManager|[<info>, [1484178...|
|   8|NetworkManager|[<info>, [1484178...|
|   9|NetworkManager|[<info>, [1484178...|
|  10|NetworkManager|[<info>, [1484178...|
|  11|NetworkManager|[<info>, [1484178...|
|  12|          dbus|[[system], Activa...|
|  13|NetworkManager|[<info>, [1484178...|
|  14|NetworkManager|[<info>, [1484178...|
|  15|      dhclient|[bound, 192.168.1...|
|  16|       systemd|[Starting, Networ...|
|  17|          dbus|[[system], Succes...|
|  18|       systemd|[Started, Network...|
|  19|              |[[not found[]]: J...|
|  20|              |[[not found[]]: J...|
+----+-----

In [ ]:
print_full

In [167]:
res_list = []

In [168]:
res_list.append((1,'wewe'))

In [122]:
res.take(3)

Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

# try: Map Reduce 

In [89]:
df = sc.textFile(inputPath)

In [90]:
rdd_map = df.map(lambda s : check_map(s, compare_folder))

In [91]:
print rdd_map.collect()

Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [84]:
rdd_map_temp = df.map(lambda s : s )

In [94]:
context = rdd_map_temp.take(1).pop()
print context

Jan 12 07:35:15 jerry-virtual-machine anacron[75735]: Job `cron.daily' terminated


In [95]:
total_score = 0
list_not_find = []
list_rdd_key = {}

In [96]:
if (len(context.split(" ")) > 4):
    if ("[" in context.split(" ")[4]):
        key_temp = context.split(" ")[4]
        keyword = key_temp.split("[")[0]
        keyword = str(keyword.strip())
        values = context.split(key_temp)[1].split(" ")

In [97]:
print values
print keyword

[u'', u'Job', u"`cron.daily'", u'terminated']
anacron


In [98]:
if('/' not in str(keyword) and ''  != keyword):
    if(str(keyword) not in list_rdd_key):
        list_rdd_key[str(keyword)] = sc.textFile(compare_folder + str(keyword))
        print list_rdd_key[str(keyword)].take(3)
        compare_text =list_rdd_key[str(keyword)]
        len_compare = compare_text.count()
        compare = compare_text.collect()

[u"(u'', 33)", u'(u" Job `cron.daily\' terminated (mailing output)", 1)', u'(u" Will run job `cron.weekly\' in 10 min.", 1)']


In [99]:
print compare

[u"(u'', 33)", u'(u" Job `cron.daily\' terminated (mailing output)", 1)', u'(u" Will run job `cron.weekly\' in 10 min.", 1)', u"(u' Anacron 2.3 started on 2017-01-08', 1)", u'(u" Updated timestamp for job `cron.daily\' to 2017-01-06", 1)', u"(u' Anacron 2.3 started on 2017-01-06', 1)", u'(u" Job `cron.weekly\' started", 1)', u'(u" Updated timestamp for job `cron.daily\' to 2017-01-08", 1)', u'(u" anacron: Can\'t find sendmail at /usr/sbin/sendmail, not mailing output", 1)', u'(u" Will run job `cron.daily\' in 5 min.", 4)', u'(u" Job `cron.daily\' terminated", 3)', u"(u' Normal exit (2 jobs run)', 1)", u'(u" Can\'t find sendmail at /usr/sbin/sendmail, not mailing output", 1)', u'(u" Updated timestamp for job `cron.daily\' to 2017-01-07", 1)', u"(u' Anacron 2.3 started on 2017-01-07', 1)", u"(u' Anacron 2.3 started on 2017-01-09', 1)", u"(u' Normal exit (1 job run)', 3)", u'(u" Job `cron.daily\' started", 4)', u'(u" Job `cron.weekly\' terminated", 1)', u'(u" Updated timestamp for job `cr

In [100]:
print values

[u'', u'Job', u"`cron.daily'", u'terminated']


In [101]:
for val in values:
    flg_found = 0
    if (val != ""):
        for i in range (1,len_compare):
            #split lines by space 
            comapre_line = compare[i].split(" ")
            for ele_com_line in comapre_line:
                #compare value from each text
                if (str(val) in str(ele_com_line)):
                    flg_found = 1
                    break;
            if(flg_found == 1):
                break;
        if(flg_found == 0):
            list_not_find.append(str(val))

In [104]:
try:
    if (len(context.split(" ")) > 4):
        if ("[" in context.split(" ")[4]):
            key_temp = context.split(" ")[4]
            keyword = key_temp.split("[")[0]
            keyword = str(keyword.strip())
            values = context.split(key_temp)[1].split(" ")
            print values
            if(str(keyword) not in list_rdd_key):
                list_rdd_key[str(keyword)] = sc.textFile(compare_folder+ str(keyword))
            for val in values:
                flg_found = 0
                if (val != ""):
                    for i in range (1,len_compare):
                        #split lines by space 
                        comapre_line = compare[i].split(" ")
                        for ele_com_line in comapre_line:
                            #compare value from each text
                            if (str(val) in str(ele_com_line)):
                                flg_found = 1
                                break;
                        if(flg_found == 1):
                            break;
                    if(flg_found == 0):
                        list_not_find.append(str(val))
        else:
            list_not_find.append("[not found[]]: " + str(context))
    else:
        list_not_find.append("[not found, less word]: " + str(context))
except:
    list_not_find.append("[error happen]: " + str(context))
    print("error happen==="+str(keyword))

[u'', u'Job', u"`cron.daily'", u'terminated']


In [93]:
rdd_map.take(2)

Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [26]:
print hasattr(rdd_map, "toDF")

False


In [205]:
a = " Job `cron.daily' terminated"

In [206]:
b = a.split(" ")

In [207]:
print b

['', 'Job', "`cron.daily'", 'terminated']


In [208]:
context = "Jan 12 07:35:15 jerry-virtual-machine anacron[75735]: Job `cron.daily' terminated"

In [209]:
if (len(context.split(" ")) > 4):
    if ("[" in context.split(" ")[4]):
        key_temp = context.split(" ")[4]
        keyword = key_temp.split("[")[0]
        keyword = str(keyword.strip())
        values = context.split(key_temp)[1].split(" ")
        #get hdfs if never used
        if(str(keyword) not in list_rdd_key):
            list_rdd_key[str(keyword)] = sc.textFile(compare_folder+ str(keyword))
        for val in values:
            flg_found = 0
            if (val != ""):
                for i in range (1,len_compare):
                    #split lines by space 
                    comapre_line = compare[i].split(" ")
                    for ele_com_line in comapre_line:
                        #compare value from each text
                        if (str(val) in str(ele_com_line)):
                            flg_found = 1
                            break;
                    if(flg_found == 1):
                        break;
                if(flg_found == 0):
                    list_not_find.append(str(val))
    else:
        list_not_find.append("[not found[]]: " + str(context))
else:
    list_not_find.append("[not found, less word]: " + str(context))
